# Libraries

In [7]:
import pyodbc
import pandas as pd
from openpyxl import load_workbook

# Database settings

In [8]:
# Connect to the database
server = r'NOLTE01\CHARLESNOLTE'
database = 'SouthAfrica_CrimeStats'
username = 'cnoltehj'
password = 'HeidiJoy@1'


# 2020-2024 Validate Policestation and Exclude rows

In [8]:
# Load the Excel file
# file_path = "./Dataset_CSV_Files/2024_2025/2024-2025_-_1st_Quarter_WEB.xlsx"
# Quarter = 1

file_path = "./Dataset_CSV_Files/2024_2025/2024-2025_-_2nd_Quarter_WEB.xlsx"
Quarter = 2

# file_path = "./Dataset_CSV_Files/2024_2025/2024_2025_3rd_Quarter_WEB.xlsx"
# Quarter = 3

# file_path = "./Dataset_CSV_Files/2024_2025/2024-2025_Q4_crime_stats.xlsx" 
# Quarter = 4

wb = load_workbook(file_path, data_only=True)

# Access the desired sheet
# Q-1, 2,3,4
sheet = wb["Crime stats per component"]

# Quarter
#sheet = wb["Crime stats per component"]

# Check the value of cell Q8
cell_l8 = sheet["L8"].value

# Check the value of cell Q9
cell_l9 = sheet["L9"].value

# Q-, 1 2,3,4
excluded_rows = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 23, 28, 36, 40, 47, 52, 54, 55, 61, 62]
excluded_columns = [1, 2, sheet.max_column - 1, sheet.max_column]  # First 2 columns and last 2 columns

print("Quarter : ",Quarter)
print("Province : ",cell_l8)
print("Police Station : ",cell_l9)


#     # Iterate over all rows except for the excluded ones
for row_number in range(2, sheet.max_row + 1):
    if row_number not in excluded_rows:
            # Retrieve row values and exclude specified columns
        row_values = [cell.value for col_num, cell in enumerate(sheet[row_number]) if col_num + 1 not in excluded_columns]

            # Print the row values as a single dataset
        print(row_values)



Quarter :  2
Province :  Western Cape
Police Station :  Albertinia
['Murder', 0, 0, 2, 0, 0, 0, '0 count diff']
['Sexual offences', 0, 3, 1, 4, 0, -4, '4 counts lower']
['Attempted murder', 0, 0, 0, 0, 1, 1, '1 count higher']
['Assault with the intent to inflict grievous bodily harm', 16, 11, 16, 19, 11, -8, '8 counts lower']
['Common assault', 21, 19, 26, 26, 22, -4, '4 counts lower']
['Common robbery', 1, 1, 0, 2, 1, -1, '1 count lower']
['Robbery with aggravating circumstances', 1, 1, 0, 2, 3, 1, '1 count higher']
['Contact crime (Crimes against the person)', 39, 35, 45, 53, 38, -15, '15 counts lower']
['Rape', 0, 2, 1, 3, 0, -3, '3 counts lower']
['Sexual assault', 0, 1, 0, 1, 0, -1, '1 count lower']
['Attempted sexual offences', 0, 0, 0, 0, 0, 0, '0 count diff']
['Contact sexual offences', 0, 0, 0, 0, 0, 0, '0 count diff']
['Carjacking', 0, 0, 0, 0, 0, 0, '0 count diff']
['Robbery at residential premises', 0, 0, 0, 0, 0, 0, '0 count diff']
['Robbery at non-residential premises', 0

c:\Users\cnolt\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


# 2020-2024 Update Stats into Database

In [19]:
cnxn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
cursor = cnxn.cursor()

# Load workbook
file_path = './Dataset_CSV_Files/2024_2025/Q4_2025.xlsx' 
wb = load_workbook(file_path, data_only=True)

# Loop through sheet1 - sheet150
for i in range(152, 153):
    sheet_name = f"Sheet{i}"
    
    if sheet_name not in wb.sheetnames:
        print(f"Skipping {sheet_name} (does not exist)")
        continue
    
    sheet = wb[sheet_name]
    print(f"\nProcessing {sheet_name}...")

    # Get header info
    cell_J9 = sheet["L8"].value
    cell_J10 = sheet["L9"].value
    selected_province = cell_J9
    selected_policestation = 'Simons Town' #cell_J10
    selected_quarter = 4

    print(f"Quarter: {selected_quarter}, Province: {selected_province}, Police Station: {selected_policestation}")

    excluded_rows = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 23, 28, 36, 40, 47, 52, 54 , 61, 62, 63] #Q2 / Q3 / Q4

    excluded_columns = [1, 2, sheet.max_column - 1, sheet.max_column]

    # Process each data row
    for row_number in range(3, sheet.max_row + 1):
        if row_number in excluded_rows:
            continue

        row_values = [
            cell.value for col_num, cell in enumerate(sheet[row_number])
            if col_num + 1 not in excluded_columns
        ]

        # Skip empty or invalid rows
        if all(v in (None, "", " ") for v in row_values):
            continue

        try:
            cursor.execute(
                "EXEC sp_Update_QuarterlyCrimeStats_2020_2024 ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?",
                (
                    str(row_values[0]).strip(),
                    row_values[1],
                    row_values[2],
                    row_values[3],
                    row_values[4],
                    row_values[5],
                    row_values[6],
                    str(row_values[7]).strip(),
                    str(selected_province).strip(),
                    str(selected_policestation).strip(),
                    int(selected_quarter),
                ),
            )
            cnxn.commit()
            print(f"Update row {row_number} from {sheet_name}")

        except pyodbc.Error as e:
            print(f"SQL Error in {sheet_name} row {row_number}: {e}")

# Cleanup
cursor.close()
cnxn.close()
wb.close()
print("\n Processing complete for all sheets.")


Processing Sheet152...
Quarter: 4, Province: Western Cape, Police Station: Simons Town
Update row 15 from Sheet152
Update row 16 from Sheet152
Update row 17 from Sheet152
Update row 18 from Sheet152
Update row 19 from Sheet152
Update row 20 from Sheet152
Update row 21 from Sheet152
Update row 22 from Sheet152
Update row 24 from Sheet152
Update row 25 from Sheet152
Update row 26 from Sheet152
Update row 27 from Sheet152
Update row 29 from Sheet152
Update row 30 from Sheet152
Update row 31 from Sheet152
Update row 32 from Sheet152
Update row 33 from Sheet152
Update row 34 from Sheet152
Update row 35 from Sheet152
Update row 37 from Sheet152
Update row 38 from Sheet152
Update row 39 from Sheet152
Update row 41 from Sheet152
Update row 42 from Sheet152
Update row 43 from Sheet152
Update row 44 from Sheet152
Update row 45 from Sheet152
Update row 46 from Sheet152
Update row 48 from Sheet152
Update row 49 from Sheet152
Update row 50 from Sheet152
Update row 51 from Sheet152
Update row 53 fr

# Test Code

In [21]:

import streamlit, sklearn, shap, lime, plotly
print("✅ All key modules imported successfully.")

✅ All key modules imported successfully.
